## Initial Data

In [2]:
import pandas as pd
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphlab as gl

In [3]:
final = pd.read_csv("final_pull.csv")
del final["Unnamed: 0"]
final.head()

,artistName,artworkUrl100,releaseDate,collectionName,decade,updated_genre,target
0,Admiral Bailey,http://is5.mzstatic.com/image/thumb/Music/v4/0...,1987-06-09T07:00:00Z,The Best of Admiral Bailey,1980.0,Reggae,Reggae-1980.0
1,Admiral Bailey,http://is3.mzstatic.com/image/thumb/Music/v4/0...,1988-01-01T08:00:00Z,Big Belly,1980.0,Reggae,Reggae-1980.0
2,Admiral Bailey,http://is4.mzstatic.com/image/thumb/Music/v4/3...,1987-06-09T07:00:00Z,Best of Admiral Bailey,1980.0,Reggae,Reggae-1980.0
3,Admiral Bailey,http://is5.mzstatic.com/image/thumb/Music/v4/2...,2006-06-24T07:00:00Z,Admiral Bailey's Turn Off the Heat - EP,2000.0,World,World-2000.0
4,Admiral Bailey,http://is5.mzstatic.com/image/thumb/Music/v4/8...,2011-06-20T07:00:00Z,Dela Move,2010.0,Reggae,Reggae-2010.0


In [3]:
genre_decade = pd.pivot_table(final, index = ["updated_genre", "decade"], values = ["artistName"], aggfunc = len).unstack()
genre_decade

artistName                     
decade                1980.0 1990.0 2000.0 2010.0
updated_genre                                    
Alternative              401   1527   6718   7961
Blues                     13    112    545    406
Country                  202   1023   4098   4931
Dance                     91    686   7278  18717
Electronic                71    966   5275   8165
HipHop                    21    266   2444   4441
Jazz                      83    329   1481   1564
Pop                      277    889   4718   7438
RB Soul                   62    267   1167   1523
Reggae                   185    633   3233   6821
Rock                     576   1945   9243  10537
Singer/Songwriter         69    224   1094   1696
World                    118    567   2150   2176

In [4]:
genre_decade.sum()

            decade
artistName  1980.0     2169
            1990.0     9434
            2000.0    49444
            2010.0    76376
dtype: int64

In [47]:
genre_decade.T.sum().sort_values()

updated_genre
Blues                 1076
RB Soul               3019
Singer/Songwriter     3083
Jazz                  3457
World                 5011
HipHop                7172
Country              10254
Reggae               10872
Pop                  13322
Electronic           14477
Alternative          16607
Rock                 22301
Dance                26772
dtype: int64

## Rebalance the data


Only use Genre and Decade, not the combined one - 15% sample for each

In [4]:
#pull 1K of each of sample

decade_list = list(final["decade"].unique())

decade_samps = []
for i in decade_list:
    decade = final[final["decade"] == i].sample(n=1000, random_state = 77)
    decade_samps.append(decade)
    
decade_sample = pd.concat(decade_samps)


In [5]:
decade_sample["decade"].value_counts()

2010.0    1000
2000.0    1000
1990.0    1000
1980.0    1000
Name: decade, dtype: int64

In [6]:
decade_sample["updated_genre"].value_counts()

Rock                 797
Alternative          580
Dance                498
Pop                  390
Electronic           380
Country              362
Reggae               305
World                166
HipHop               156
Jazz                 134
RB Soul              110
Singer/Songwriter     88
Blues                 34
Name: updated_genre, dtype: int64

In [7]:
#pull 800 of each of sample

genre_list = list(final["updated_genre"].unique())

genre_samps = []
for i in genre_list:
    genre = final[final["updated_genre"] == i].sample(n=800, random_state = 77)
    genre_samps.append(genre)
genre_sample = pd.concat(genre_samps)

In [8]:
genre_sample["updated_genre"].value_counts()

Country              800
Electronic           800
Blues                800
RB Soul              800
Rock                 800
Jazz                 800
Dance                800
Pop                  800
Reggae               800
Singer/Songwriter    800
World                800
HipHop               800
Alternative          800
Name: updated_genre, dtype: int64

In [9]:
genre_sample["decade"].value_counts()

2010.0    5421
2000.0    3962
1990.0     839
1980.0     178
Name: decade, dtype: int64

### 2. Begin to train test split for each

In [10]:
y_genre = genre_sample["updated_genre"]
y_decade = decade_sample["decade"]

In [11]:
y_genre.index

Int64Index([ 91338, 101107,  12564,   5415,   4449,   3550,  41918,   5324,
              3849, 101913,
            ...
             45215, 126428, 107491,  14989,  85675, 129945,  51417, 120238,
            116312, 111723],
           dtype='int64', length=10400)

In [12]:
X_g_paths = ["./image/"+ str(i+1)+".jpg" for i in y_genre.index]
X_d_paths = ["./image/"+ str(i+1)+".jpg" for i in y_decade.index]

In [13]:
#make sure this is right
X_g_paths[25]

'./image/1217.jpg'

In [14]:
y_genre.iloc[25]

'Reggae'

In [15]:
genre_sample.iloc[25]

artistName                 Beenie Man, Bounty Killer & Sly & Robbie
artworkUrl100     http://is4.mzstatic.com/image/thumb/Music/v4/7...
releaseDate                                    2011-06-14T07:00:00Z
collectionName    Sly & Robbie presents Beenie \ Bounty: The Tax...
decade                                                         2010
updated_genre                                                Reggae
target                                                Reggae-2010.0
Name: 1216, dtype: object

In [16]:
from sklearn.cross_validation import train_test_split
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g_paths, y_genre, test_size = .20)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d_paths, y_decade, test_size = .20)

### 3. Attempt to fit data with Neural Nets using Graph Lab

In [17]:
X_train_g_sf = [gl.Image(i) for i in X_train_g]
X_train_g_sf = [gl.image_analysis.resize(i, 100, 100, 3, decode=True) for i in X_train_g_sf]
X_train_g_sf = gl.SFrame(X_train_g_sf)
X_train_g_sf["labels"] = y_train_g
print X_train_g_sf

This non-commercial license of GraphLab Create for academic use is assigned to katty.lein@gmail.com and will expire on August 13, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1471400353.log


+------------------------+-------------+
|           X1           |    labels   |
+------------------------+-------------+
| Height: 100 Width: 100 | Alternative |
| Height: 100 Width: 100 |  Electronic |
| Height: 100 Width: 100 |  Electronic |
| Height: 100 Width: 100 |     Jazz    |
| Height: 100 Width: 100 |  Electronic |
| Height: 100 Width: 100 |    Reggae   |
| Height: 100 Width: 100 |     Rock    |
| Height: 100 Width: 100 |    HipHop   |
| Height: 100 Width: 100 |    Reggae   |
| Height: 100 Width: 100 |    Reggae   |
+------------------------+-------------+
[8320 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [18]:
X_test_g_sf = [gl.Image(i) for i in X_test_g]
X_test_g_sf = [gl.image_analysis.resize(i, 100, 100, 3, decode=True) for i in X_test_g_sf]
X_test_g_sf = gl.SFrame(X_test_g_sf)
X_test_g_sf["labels"] = y_test_g
print X_test_g_sf

+------------------------+-------------+
|           X1           |    labels   |
+------------------------+-------------+
| Height: 100 Width: 100 |     Pop     |
| Height: 100 Width: 100 |     Jazz    |
| Height: 100 Width: 100 |   RB Soul   |
| Height: 100 Width: 100 |     Rock    |
| Height: 100 Width: 100 |   Country   |
| Height: 100 Width: 100 |    HipHop   |
| Height: 100 Width: 100 | Alternative |
| Height: 100 Width: 100 |     Rock    |
| Height: 100 Width: 100 |     Rock    |
| Height: 100 Width: 100 |     Pop     |
+------------------------+-------------+
[2080 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [152]:
net_g = gl.deeplearning.create(X_train_g_sf[["X1", "labels"]], target="labels")

In [153]:
nn_model_g = gl.neuralnet_classifier.create(X_train_g_sf[["X1", "labels"]], target='labels', network = net_g, metric=['accuracy', 'recall@2'], max_iterations=3)

Using network:

### network layers ###
layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 2
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[4]: RectifiedLinearLayer
layer[5]: DropoutLayer
  threshold = 0.5
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 13
layer[7]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Computing mean image...

Done computing mean image.

Creating neuralnet using cpu

Training with batch size = 100

...

| 3         | 3400     | 121.615506   | 0.104118          |                     | 0.199118          |                     | 163.448563      |

| 3         | 5100     | 132.159414   | 0.106863          |                     | 0.199608          |                     | 161.230530      |

| 3         | 6800     | 142.555707   | 0.110588          |                     | 0.204853          |                     | 163.519836      |

| 3         | 8000     | 150.942017   | 0.110500          | 0.085642            | 0.203125          | 0.183879            | 143.090271      |

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

In [154]:
eval_g_sf = nn_model_g.evaluate(X_test_g_sf)

In [159]:
#performed very poorly
eval_g_sf

{'accuracy': 0.11730769276618958, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 165
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 | Alternative  |   Alternative   |   29  |
 |    Blues     |   Alternative   |   12  |
 |   Country    |   Alternative   |   14  |
 |    Dance     |   Alternative   |   12  |
 |  Electronic  |   Alternative   |   21  |
 |    HipHop    |   Alternative   |   13  |
 |     Jazz     |   Alternative   |   19  |
 |     Pop      |   Alternative   |   14  |
 |   RB Soul    |   Alternative   |   8   |
 |    Reggae    |   Alternative   |   9   |
 +--------------+-----------------+-------+
 [165 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [19]:
X_train_d_sf = [gl.Image(i) for i in X_train_d]
X_train_d_sf = [gl.image_analysis.resize(i, 100, 100, 3, decode=True) for i in X_train_d_sf]
X_train_d_sf = gl.SFrame(X_train_d_sf)
X_train_d_sf["labels"] = y_train_d

In [20]:
X_test_d_sf = [gl.Image(i) for i in X_test_d]
X_test_d_sf = [gl.image_analysis.resize(i, 100, 100, 3, decode=True) for i in X_test_d_sf]
X_test_d_sf = gl.SFrame(X_test_d_sf)
X_test_d_sf["labels"] = y_test_d

In [21]:
X_test_d_sf["labels"] = X_test_d_sf["labels"].apply(lambda x: int(x))
X_train_d_sf["labels"] = X_train_d_sf["labels"].apply(lambda x: int(x))

In [127]:
net_d = gl.deeplearning.create(X_train_d_sf[["X1", "labels"]], target="labels")

In [128]:
net_d.layers

layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 2
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[4]: RectifiedLinearLayer
layer[5]: DropoutLayer
  threshold = 0.5
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 4
layer[7]: SoftmaxLayer

In [139]:
#did slightly better at 50
net_d.layers[3].num_hidden_units = 50

In [140]:
nn_model_d = gl.neuralnet_classifier.create(X_train_d_sf[["X1", "labels"]], target='labels', network = net_d, metric=['accuracy', 'recall@2'], max_iterations=3)

Using network:

### network layers ###
layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 2
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 50
layer[4]: RectifiedLinearLayer
layer[5]: DropoutLayer
  threshold = 0.5
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 4
layer[7]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Computing mean image...

Done computing mean image.

Creating neuralnet using cpu

Training with batch size = 100

...

| 2         | 1800     | 28.550208    | 0.275000          |                     | 0.532778          |                     | 175.583176      |

| 2         | 3100     | 36.311898    | 0.283226          | 0.309091            | 0.539032          | 0.539394            | 167.489212      |

| 3         | 1800     | 46.522040    | 0.283333          |                     | 0.532222          |                     | 176.497528      |

| 3         | 3100     | 54.285828    | 0.287419          | 0.260606            | 0.537742          | 0.600000            | 167.443939      |

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

In [142]:
eval_d_sf = nn_model_d.evaluate(X_test_d_sf)

In [143]:
#best performing so far but only predicting 3 values
eval_d_sf

{'accuracy': 0.29875001311302185, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     1980     |       1980      |   71  |
 |     1990     |       1980      |   68  |
 |     2000     |       1980      |   46  |
 |     2010     |       1980      |   49  |
 |     1980     |       1990      |   14  |
 |     1990     |       1990      |   16  |
 |     2000     |       1990      |   11  |
 |     2010     |       1990      |   12  |
 |     1980     |       2000      |   31  |
 |     1990     |       2000      |   40  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

### 4. Feature Extraction

deep feature extractor 

In [22]:
extractor = gl.feature_engineering.DeepFeatureExtractor(features = 'X1', model='auto')

Downloading https://static.turi.com/products/graphlab-create/resources/models/python2.7/imagenet_model_iter45/dir_archive.ini to /var/tmp/graphlab-katty/2559/1bab91a8-879a-46a3-a95f-6ed5c0a884e2.ini

Downloading https://static.turi.com/products/graphlab-create/resources/models/python2.7/imagenet_model_iter45/objects.bin to /var/tmp/graphlab-katty/2559/7a7f5edd-a8ad-487b-98f0-ea6d509ef033.bin

In [23]:
extractor_g_train = extractor.fit(X_train_g_sf)
features_g_train = extractor_g_train.transform(X_train_g_sf)
features_g_train

Images being resized.

X1,labels,deep_features.X1
Height: 100 Width: 100,Alternative,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 3.14219546318, 0.0, ..."
Height: 100 Width: 100,Electronic,"[0.0, 0.0210257172585,1.28963172436, 0.0, 0.0, ..."
Height: 100 Width: 100,Electronic,"[0.0, 0.0, 0.0,2.37297391891, ..."
Height: 100 Width: 100,Jazz,"[0.0, 4.92607307434, 0.0,0.0, 0.0, 0.0, ..."
Height: 100 Width: 100,Electronic,"[0.0, 0.0, 0.0, 0.0, 0.0,0.13795119524, ..."
Height: 100 Width: 100,Reggae,"[0.0, 0.0, 0.0, 0.0, 0.0,3.0610909462, ..."
Height: 100 Width: 100,Rock,"[0.0, 0.0, 0.0, 0.0,1.22484409809, ..."
Height: 100 Width: 100,HipHop,"[0.0, 1.95456123352,0.49214309454, 0.0, ..."
Height: 100 Width: 100,Reggae,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 3.9449672699, ..."
Height: 100 Width: 100,Reggae,"[0.0, 0.0, 0.0, 0.0,0.422669410706, ..."


In [83]:
extractor_g_train.summary()

Class                : DeepFeatureExtractor

Model fields
------------
Feature(s)           : ['X1']
Output column prefix : deep_features



In [24]:
extractor_g_test = extractor.fit(X_test_g_sf)
features_g_test = extractor_g_test.transform(X_test_g_sf)
features_g_test

Images being resized.

X1,labels,deep_features.X1
Height: 100 Width: 100,Pop,"[0.0, 0.0, 0.0, 0.0,2.71038007736, 0.0, ..."
Height: 100 Width: 100,Jazz,"[0.0, 0.0, 0.0,1.16559708118, ..."
Height: 100 Width: 100,RB Soul,"[0.0, 0.0, 0.0, 0.0,0.440298557281, ..."
Height: 100 Width: 100,Rock,"[1.43266046047,0.849198520184, ..."
Height: 100 Width: 100,Country,"[0.0, 3.45019960403, 0.0,0.0, 0.0, 0.316090643 ..."
Height: 100 Width: 100,HipHop,"[0.0, 0.0, 0.0,0.360220193863, 0.0, ..."
Height: 100 Width: 100,Alternative,"[0.0, 1.16141164303, 0.0,0.0, 0.0, 0.0, ..."
Height: 100 Width: 100,Rock,"[0.0, 1.4160169363, 0.0,0.0, 0.0, 0.1148660779, ..."
Height: 100 Width: 100,Rock,"[0.545539796352, 0.0,0.0658822655678, 0.0, ..."
Height: 100 Width: 100,Pop,"[0.0, 0.0, 0.0, 0.0, 0.0,1.31371116638, ..."


In [25]:
#set up deep feature extraction for decades as well
extractor_d_train = extractor.fit(X_train_d_sf)
features_d_train = extractor_d_train.transform(X_train_d_sf)

Images being resized.

In [31]:
features_d_train.export_csv("features_d_train.csv")

In [32]:
features_g_test.export_csv("features_g_test.csv")
features_g_train.export_csv("features_g_train.csv")

In [34]:
extractor_d_test = extractor.fit(X_test_d_sf)

features_d_test = extractor_d_test.transform(X_test_d_sf)

Images being resized.

In [35]:
features_d_test.export_csv("features_d_test.csv")

try to extract features from the previous neural net

In [90]:
X_train_g_sf['feat_test'] = nn_model_g.extract_features(X_train_g_sf)

In [155]:
X_train_d_sf['feat_test'] = nn_model_d.extract_features(X_train_d_sf)

In [94]:
net_g_test = gl.deeplearning.create(X_test_g_sf, target="labels")
nn_model_g_test = gl.neuralnet_classifier.create(X_test_g_sf, target='labels', network = net_g_test, metric=['accuracy', 'recall@2'], max_iterations=3)

Using network:

### network layers ###
layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 2
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[4]: RectifiedLinearLayer
layer[5]: DropoutLayer
  threshold = 0.5
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 13
layer[7]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Computing mean image...

Done computing mean image.

Creating neuralnet using cpu

Training with batch size = 100

...

| 2         | 1600     | 23.708863    | 0.100000          |                     | 0.183125          |                     | 157.083542      |

| 2         | 2000     | 26.806397    | 0.098500          | 0.101852            | 0.181500          | 0.203704            | 129.134857      |

| 3         | 1700     | 37.386610    | 0.090000          |                     | 0.172941          |                     | 160.678879      |

| 3         | 2000     | 39.879027    | 0.091500          | 0.083333            | 0.173500          | 0.166667            | 120.364922      |

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

In [157]:
net_d_test = gl.deeplearning.create(X_test_d_sf[["X1", "labels"]], target="labels")

In [158]:
net_d_test.layers[3].num_hidden_units = 50

In [159]:
nn_model_d_test = gl.neuralnet_classifier.create(X_test_d_sf[["X1", "labels"]], target='labels', network = net_d_test, metric=['accuracy', 'recall@2'], max_iterations=3)

Using network:

### network layers ###
layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 2
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 50
layer[4]: RectifiedLinearLayer
layer[5]: DropoutLayer
  threshold = 0.5
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 4
layer[7]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Computing mean image...

Done computing mean image.

Creating neuralnet using cpu

Training with batch size = 100

...

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

| 1         | 800      | 5.137082     | 0.258750          | 0.235294            | 0.511250          | 0.392157            | 155.730438      |

| 2         | 800      | 9.833789     | 0.297500          | 0.235294            | 0.546250          | 0.470588            | 170.336517      |

| 3         | 800      | 14.579892    | 0.341250          | 0.235294            | 0.588750          | 0.372549            | 168.562775      |

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

In [160]:
X_test_g_sf['feat_test'] = nn_model_g_test.extract_features(X_test_g_sf)
X_test_d_sf['feat_test'] = nn_model_d_test.extract_features(X_test_d_sf)

### None of this working! Why don't we try to fit our own neural network?'

In [163]:
#create mlp for decades
percpt_net_d = gl.deeplearning.MultiLayerPerceptrons(num_hidden_layers=3,
                                                num_hidden_units=[50,10,4])

In [164]:
#create convolutionnet for decades
conv_net_d = gl.deeplearning.ConvolutionNet(num_convolution_layers=2,
                                              kernel_size=3,
                                              num_channels=10,
                                              num_output_units=4)

In [165]:
#i dont know what im doinggggggg
conv_net_d = gl.deeplearning.ConvolutionNet(num_convolution_layers=2,
                                              kernel_size=3,
                                              num_channels=10,
                                              num_output_units=0)


In [167]:
conv_net_d.layers

layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 1
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 1
  kernel_size = 3
layer[2]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 1
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[3]: MaxPoolingLayer
  padding = 0
  stride = 1
  kernel_size = 3
layer[4]: FlattenLayer

In [168]:
conv_net_d.layers.extend(percpt_net_d.layers)

In [169]:
conv_net_d.layers

layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 1
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 1
  kernel_size = 3
layer[2]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 1
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[3]: MaxPoolingLayer
  padding = 0
  stride = 1
  kernel_size = 3
layer[4]: FlattenLayer
layer[5]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 50
layer[6]: SigmoidLayer
layer[7]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[8]: SigmoidLayer
layer[9]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 4
layer[10]: SoftmaxLayer

In [171]:
#fit decades 
nettest_d = gl.deeplearning.create(X_train_d_sf[["X1", "labels"]], target="labels")
nn_nnettest_d = gl.neuralnet_classifier.create(X_train_d_sf[["X1", "labels"]], target='labels', network = nettest_d, metric=['accuracy', 'recall@2'], max_iterations=3)

Using network:

### network layers ###
layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 2
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 2
  kernel_size = 3
layer[2]: FlattenLayer
layer[3]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[4]: RectifiedLinearLayer
layer[5]: DropoutLayer
  threshold = 0.5
layer[6]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 4
layer[7]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Computing mean image...

Done computing mean image.

Creating neuralnet using cpu

Training with batch size = 100

...

| 2         | 1800     | 28.740949    | 0.275000          |                     | 0.545556          |                     | 175.763687      |

| 2         | 3100     | 36.546628    | 0.278387          | 0.247059            | 0.534839          | 0.505882            | 166.545319      |

| 3         | 1800     | 46.618211    | 0.299444          |                     | 0.562778          |                     | 178.722595      |

| 3         | 3100     | 54.414823    | 0.286452          | 0.288235            | 0.550323          | 0.594118            | 166.739014      |

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

In [172]:
#did not perform that much better than the other neural network
nn_nnettest_d.evaluate(X_test_d_sf)

{'accuracy': 0.27000001072883606, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     1980     |       1980      |   36  |
 |     1990     |       1980      |   36  |
 |     2000     |       1980      |   27  |
 |     2010     |       1980      |   16  |
 |     1980     |       1990      |   19  |
 |     1990     |       1990      |   17  |
 |     2000     |       1990      |   15  |
 |     2010     |       1990      |   16  |
 |     1980     |       2000      |   90  |
 |     1990     |       2000      |   96  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [222]:
#try one a few other neural nets. increase hidden layers

percpt_net_d = gl.deeplearning.MultiLayerPerceptrons(num_hidden_layers=6,
                                                num_hidden_units=[200,150, 100, 50,30,4])

#change output units to exactly 4
conv_net_d = gl.deeplearning.ConvolutionNet(num_convolution_layers=2,
                                              kernel_size=3,
                                              num_channels=10,
                                              num_output_units=0)


conv_net_d.layers.extend(percpt_net_d.layers)

In [223]:
conv_net_d.layers

layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 1
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 1
  kernel_size = 3
layer[2]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 1
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[3]: MaxPoolingLayer
  padding = 0
  stride = 1
  kernel_size = 3
layer[4]: FlattenLayer
layer[5]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 200
layer[6]: SigmoidLayer
layer[7]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 150
layer[8]: SigmoidLayer
layer[9]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[10]: SigmoidLayer
layer[11]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 50
layer[12]: SigmoidLayer
layer[13]

In [224]:
nettest_d = gl.deeplearning.create(X_train_d_sf[["X1", "labels"]], target="labels")
nn_nnettest_d = gl.neuralnet_classifier.create(X_train_d_sf[["X1", "labels"]], target='labels', network = conv_net_d, metric=['accuracy', 'recall@2'], max_iterations=3)

Using network:

### network layers ###
layer[0]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 1
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[1]: MaxPoolingLayer
  padding = 0
  stride = 1
  kernel_size = 3
layer[2]: ConvolutionLayer
  init_random = gaussian
  padding = 0
  stride = 1
  num_channels = 10
  num_groups = 1
  kernel_size = 3
layer[3]: MaxPoolingLayer
  padding = 0
  stride = 1
  kernel_size = 3
layer[4]: FlattenLayer
layer[5]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 200
layer[6]: SigmoidLayer
layer[7]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 150
layer[8]: SigmoidLayer
layer[9]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 100
layer[10]: SigmoidLayer
layer[11]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units

Computing mean image...

Done computing mean image.

Creating neuralnet using cpu

Training with batch size = 100

...

| 3         | 2900     | 1180.012661  | 0.251034          |                     | 0.492414          |                     | 7.879230        |

| 3         | 3000     | 1192.688191  | 0.249667          |                     | 0.492667          |                     | 7.889216        |

| 3         | 3100     | 1205.472060  | 0.247097          |                     | 0.490000          |                     | 7.822358        |

| 3         | 3100     | 1216.128330  | 0.247097          | 0.254658            | 0.490000          | 0.490683            | 7.822358        |

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

In [225]:
nn_nnettest_d.evaluate(X_test_d_sf)

{'accuracy': 0.24375000596046448, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     1980     |       2010      |  199  |
 |     1990     |       2010      |  210  |
 |     2000     |       2010      |  196  |
 |     2010     |       2010      |  195  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

This doesn't seem too successful for decade only... might be better to stick with original nn.

Also had tried to run this for with a few variations that aren't seen here - 1 convolution net vs 2, 7 hidden layers vs 5,4,3,2 with different hidden units.

### 5. KNN

try extracted features from nn

In [98]:
knn_g_ft = gl.nearest_neighbor_classifier.create(X_train_g_sf[["labels", "feat_test"]], "labels")

Starting ball tree nearest neighbors model training.

+------------+--------------+

| Tree level | Elapsed Time |

+------------+--------------+

| 0          | 109.852ms    |

| 1          | 197.157ms    |

| 2          | 265.954ms    |

| 3          | 333.45ms     |

| 4          | 367.421ms    |

+------------+--------------+

In [103]:
knn_g_ft.evaluate(X_test_g_sf[["labels", "feat_test"]], max_neighbors = 25)

+--------------+-------------+--------------+

| Query points | % Complete. | Elapsed Time |

+--------------+-------------+--------------+

| 1            | 0           | 140.631ms    |

| 33           | 1.5         | 1.21s        |

...

| 2033         | 97.5        | 54.22s       |

| 2071         | 99.5        | 55.25s       |

| Done         |             | 55.46s       |

+--------------+-------------+--------------+

{'accuracy': 0.08701923076923077, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 168
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |    World     |       Jazz      |   3   |
 |    Reggae    |   Alternative   |   15  |
 |    Reggae    |    Electronic   |   6   |
 |    HipHop    |      Reggae     |   1   |
 |   Country    |     Country     |   7   |
 | Alternative  |       Pop       |   17  |
 |  Electronic  |       Jazz      |   1   |
 |   Country    |      World      |   4   |
 | Alternative  |      Dance      |   3   |
 |    Reggae    |      Blues      |   18  |
 +--------------+-----------------+-------+
 [168 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [161]:
knn_d_ft = gl.nearest_neighbor_classifier.create(X_train_d_sf[["labels", "feat_test"]], "labels")

Starting ball tree nearest neighbors model training.

+------------+--------------+

| Tree level | Elapsed Time |

+------------+--------------+

| 0          | 21.192ms     |

| 1          | 46.537ms     |

| 2          | 56.969ms     |

+------------+--------------+

In [162]:
knn_d_ft.evaluate(X_test_d_sf[["labels", "feat_test"]], max_neighbors = 200)

+--------------+-------------+--------------+

| Query points | % Complete. | Elapsed Time |

+--------------+-------------+--------------+

| 1            | 0           | 40.455ms     |

| 132          | 16.5        | 1.05s        |

...

| 611          | 76.25       | 5.04s        |

| 729          | 91          | 6.06s        |

| Done         |             | 6.68s        |

+--------------+-------------+--------------+

{'accuracy': 0.2825, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     2000     |       2010      |   46  |
 |     1980     |       2000      |   21  |
 |     2000     |       2000      |   28  |
 |     2010     |       2000      |   32  |
 |     2010     |       2010      |   55  |
 |     1980     |       2010      |   40  |
 |     1990     |       2000      |   19  |
 |     1990     |       2010      |   47  |
 |     1990     |       1980      |   68  |
 |     1980     |       1980      |   67  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

try deep features 

In [36]:
knn_g = gl.nearest_neighbor_classifier.create(features_g_train[["labels", "deep_features.X1"]], "labels")

Starting brute force nearest neighbors model training.

In [49]:
#100 performing the best at .19
knn_g_eval = knn_g.evaluate(features_g_test[["labels", "deep_features.X1"]], max_neighbors = 100)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 2080         | 4326400 | 25          | 10.87s       |

| Done         | 1.7e+07 | 100         | 11.04s       |

+--------------+---------+-------------+--------------+

In [79]:
knn_g_eval['confusion_matrix'].print_rows(num_rows=162, num_columns = 3)
knn_g_eval["accuracy"]

+-------------------+-------------------+-------+
|    target_label   |  predicted_label  | count |
+-------------------+-------------------+-------+
|    Alternative    |        Jazz       |   4   |
|       Reggae      |        Jazz       |   2   |
|      Country      |    Alternative    |   2   |
|        Jazz       |       Reggae      |   6   |
|       HipHop      |       Reggae      |   8   |
| Singer/Songwriter |        Rock       |   11  |
|    Alternative    |       Reggae      |   9   |
|       Blues       |    Alternative    |   2   |
|      Country      |        Rock       |   5   |
|       World       |        Jazz       |   7   |
|      Country      |       Dance       |   10  |
|       Reggae      |     Electronic    |   3   |
|       Reggae      |    Alternative    |   4   |
|       Dance       |      Country      |   12  |
|     Electronic    |        Jazz       |   10  |
|       Blues       |        Pop        |   3   |
| Singer/Songwriter |       Dance       |   14  |


0.18509615384615385

In [51]:
#fit decade
knn_d = gl.nearest_neighbor_classifier.create(features_d_train[["labels", "deep_features.X1"]], "labels")

Starting brute force nearest neighbors model training.

In [66]:
#125 - .34
knn_d_eval = knn_d.evaluate(features_d_test[["labels", "deep_features.X1"]], max_neighbors = 125)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 800          | 640000  | 25          | 1.93s        |

| Done         | 2560000 | 100         | 1.99s        |

+--------------+---------+-------------+--------------+

In [69]:
knn_d_eval["confusion_matrix"].print_rows(num_rows = 16, num_columns =3)
knn_d_eval["accuracy"]

+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|     2010     |       2010      |   42  |
|     1980     |       2010      |   11  |
|     2000     |       2010      |   29  |
|     2000     |       2000      |   52  |
|     1990     |       1990      |   87  |
|     2010     |       1990      |   56  |
|     1980     |       1980      |   95  |
|     2000     |       1980      |   44  |
|     1990     |       1980      |   79  |
|     1990     |       2010      |   16  |
|     1990     |       2000      |   28  |
|     2010     |       1980      |   57  |
|     1980     |       1990      |   58  |
|     2000     |       1990      |   71  |
|     1980     |       2000      |   35  |
|     2010     |       2000      |   40  |
+--------------+-----------------+-------+
[16 rows x 3 columns]



0.345

both performed better than the neural net but the decade works the best. Also works better than the extracted features from the neural net
https://rpubs.com/tsopronyuk/image_retrieval_system

### 6. Try Trees

In [70]:
boosted_g = gl.boosted_trees_classifier.create(features_g_train[["labels", "deep_features.X1"]], "labels")

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 7905

Number of classes           : 13

...

| 7         | 107.551284   | 0.845288          | 0.202410            | 1.347655          | 2.415859            |

| 8         | 120.570920   | 0.873877          | 0.202410            | 1.252656          | 2.415396            |

| 9         | 133.638434   | 0.893232          | 0.212048            | 1.171565          | 2.409308            |

| 10        | 146.418640   | 0.911954          | 0.212048            | 1.092228          | 2.403471            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [71]:
#performs even better
boosted_g.evaluate(features_g_test[["labels", "deep_features.X1"]])

{'accuracy': 0.23221153846153847,
 'auc': 0.6811976232910756,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 169
 
 Data:
 +-------------------+-----------------+-------+
 |    target_label   | predicted_label | count |
 +-------------------+-----------------+-------+
 |        Pop        |       Rock      |   9   |
 | Singer/Songwriter |       Pop       |   8   |
 | Singer/Songwriter |      Reggae     |   14  |
 |      Country      |       Rock      |   6   |
 |       Blues       |     Country     |   19  |
 |        Rock       |      Reggae     |   9   |
 |       Blues       |      HipHop     |   10  |
 |       HipHop      |     RB Soul     |   8   |
 |      Country      |       Jazz      |   11  |
 |      RB Soul      |      World      |   5   |
 +-------------------+-----------------+-------+
 [169 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and co

In [227]:
boosted_g_ft = gl.boosted_trees_classifier.create(X_train_g_sf[["labels", "feat_test"]], "labels")

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 7881

Number of classes           : 13

...

| 4         | 1.719923     | 0.414161          | 0.145786            | 2.205986          | 2.521192            |

| 5         | 2.137828     | 0.443979          | 0.157175            | 2.144808          | 2.513364            |

| 6         | 2.523328     | 0.473671          | 0.157175            | 2.085233          | 2.507434            |

| 10        | 3.993910     | 0.556275          | 0.173121            | 1.896286          | 2.487777            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [228]:
boosted_g_ft.evaluate(X_test_g_sf[["labels", "feat_test"]])

{'accuracy': 0.09326923076923077,
 'auc': 0.5290294546926281,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 160
 
 Data:
 +-------------------+-------------------+-------+
 |    target_label   |  predicted_label  | count |
 +-------------------+-------------------+-------+
 |       Blues       |       Dance       |   2   |
 |    Alternative    |      RB Soul      |   17  |
 | Singer/Songwriter | Singer/Songwriter |   33  |
 |       HipHop      |      Country      |   11  |
 |       Reggae      |       Reggae      |   16  |
 |        Pop        |       Reggae      |   10  |
 |        Pop        |      Country      |   8   |
 |       World       |        Jazz       |   14  |
 |       HipHop      |        Pop        |   5   |
 |       Dance       |       HipHop      |   1   |
 +-------------------+-------------------+-------+
 [160 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=

In [72]:
boosted_d = gl.boosted_trees_classifier.create(features_d_train[["labels", "deep_features.X1"]], "labels")

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 3048

...

| 7         | 10.927097    | 0.923885          | 0.315789            | 0.785983          | 1.361897            |

| 8         | 12.492684    | 0.942585          | 0.309211            | 0.737956          | 1.354532            |

| 9         | 14.054189    | 0.952100          | 0.335526            | 0.701278          | 1.360740            |

| 10        | 15.596484    | 0.958661          | 0.315789            | 0.666898          | 1.360248            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [73]:
#only marginally better
boosted_d.evaluate(features_d_test[["labels", "deep_features.X1"]])

{'accuracy': 0.35625, 'auc': 0.626174041990267, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     2010     |       1990      |   45  |
 |     1980     |       1980      |  100  |
 |     2000     |       1980      |   42  |
 |     1980     |       1990      |   37  |
 |     1980     |       2000      |   35  |
 |     2000     |       1990      |   48  |
 |     1990     |       1990      |   74  |
 |     2000     |       2010      |   52  |
 |     1980     |       2010      |   27  |
 |     2010     |       2010      |   57  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.3529672260283553, 'log_loss': 1.3226284896007514, 'precision': 0.

In [229]:
boosted_d_ft = gl.boosted_trees_classifier.create(X_train_d_sf[["labels", "feat_test"]], "labels")

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 3055

Number of classes           : 4

...

| 3         | 0.180344     | 0.624877          | 0.234483            | 1.208718          | 1.395284            |

| 4         | 0.260437     | 0.663175          | 0.248276            | 1.165759          | 1.382737            |

| 5         | 0.311499     | 0.678560          | 0.248276            | 1.129140          | 1.380055            |

| 6         | 0.365777     | 0.718167          | 0.268966            | 1.082809          | 1.383987            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [230]:
boosted_d_ft.evaluate(X_test_d_sf[["labels", "feat_test"]])

{'accuracy': 0.28875, 'auc': 0.5494479126395063, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     2000     |       2010      |   41  |
 |     1980     |       2010      |   41  |
 |     2010     |       2010      |   49  |
 |     2000     |       2000      |   58  |
 |     2010     |       2000      |   47  |
 |     1990     |       1980      |   38  |
 |     1990     |       2010      |   35  |
 |     1990     |       2000      |   56  |
 |     2000     |       1980      |   44  |
 |     1980     |       2000      |   52  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.2848579321936689, 'log_loss': 1.4095867754821014, 'precision': 0

Boosted trees does perform better than knn but its not by much

### Pause for a summary

what i did:
- rebalanced sample
- fit neural net with slightly adjusted hidden units (showed great improvement for neural nets decades)
- used deep feature extraction for features
- used feature extraction from neural nets
- fitted a custom neural net (only on decade)

#### genre
- boosted trees deep features genre = .23
- knn deep features genre = .185
- neural nets genre = .12
- knn extracted features genre = .09
- boosted trees extracted features genre = .09

#### decade
- boosted trees deep features decade = .36
- knn deep features decade = .345
- neural nets decade = .30
- boosted trees extracted features decade = .29
- knn extracted features decade = .28
- custom neural net decade = .27 (the best performing)

### 7. SVM 

Try to do one to all approach for decade

In [ ]:
svm_

### 8. Clustering